In [1]:
import numpy as np
from numpy import *
import sympy
from sympy import *
import sympy as sym
init_printing(use_unicode=True)
from sympy.physics.quantum import TensorProduct as tp
from sympy.physics.quantum import Ket, Bra
import math
from itertools import product, combinations
from functools import reduce
from IPython.core.display import HTML
import copy

# Abbreviations

- QLG: quantum logic gates
- CQLG: controlled quantum logic gates

# A simpler way to put the expression result in a more elegant and eye-pleasing form

In [ ]:
def mysim(expr):
    return sympy.simplify(expr, rational=True)

# Auxiliary functions

In [14]:
def inner_product(v,w):
    d = len(v); ip = 0
    for j in range(0,d):
        ip += conjugate(v[j])*w[j]
    return ip
#a,b,c,d = symbols("a b c d"); v = [b,a]; w = [c,d]; inner_product(v,w)

In [12]:
def norm(v):
    v = inner_product(v,v)
    return sqrt(v)
#v = [2,2]; norm(v)

In [ ]:
def cb(n,j):
    """
    Args:
        n (int): levels
        j (int): element that will be non-zero

    Returns:
        array: returns a standard base vector of C^n
    """
    vec = zeros(n,1)
    vec[j] = 1
    return vec

In [17]:
def proj(psi): 
    '''returns the projector in the psi vector'''
    d = psi.shape[0]
    P = zeros(d,d)
    for j in range(0,d):
        for k in range(0,d):
            P[j,k] = psi[j]*conjugate(psi[k])
    return P
#proj(cb(2,0))

In [13]:
def tsco(A):
    '''returns the conjugate transpose'''
    d = A.shape[0]
    new = zeros(d)
    new = sympy.transpose(A)
    new = sympy.conjugate(new)
    return Matrix(new)
#tsco(cb(2,0))

# Generic vector state

In [13]:
def psi_c(n, word='c', mtype=0):
    """
    Args:
        n (int): state vector levels
        mtype (int): array type, column array or row array. Zero (0) for
            row matrix and one (1) for row matrix

    Returns:
        array: Returns a row or column array depending on the 
               type (tp) selected with n levels. 
    """
    # Calculate the number of digits in n
    num_digits = len(str(n-1))
    # Create a list of symbolic variables with names 'c00' to 'cnn'
    psi = sympy.symbols(['{}{:0{}}'.format(word, i, num_digits) for i in range(n)])
    # Create a numpy array of zeros with size n x 1
    A = zeros(n,1)
    # Set the first n elements of A to the psi symbols
    for j in range(0,n):
        A[j] = psi[j]
    # Return the resulting array A
    if mtype == 1:
        return transpose(A).as_mutable()
    return A

In [15]:
"""
Same situation as the function above, but write the coefficients 
with indices in binary form. Useful to verify some patterns after 
the actions of certain ports like CNOT among other utilities.
"""
def psi_bin(n, word='c', mtype=0):
    num_digits = len(str(n))
    A = zeros(n,1)
    psi = [sympy.symbols('{}_{}'.format(word, format(i, '0{}b'.format(int(math.ceil(log(n, 2))))))) for i in range(n)]
    for j in range(0,n):
        A[j] = psi[j]
    if mtype == 1:
        return transpose(A).as_mutable()
    return A

In [1]:
"""
Interesting function to say exactly which matrix values
are different from zero and at the same time have a visualization
of the state vector as if it were written in Dirac notation
"""
def pbk(seq, dim=2, mtype=0):
    """
    Args:
        seq (str): input sequence. Ex: '010110'
        dim (int): computational basis dimension (default=2 for qubits)
        mtype (int): array type, column array or row array. Zero (default=0) for
            row matrix and one (1) for row matrix

    Returns:
        array: ket vector for the input sequence in the computational basis of dimension 'dim'
    """
    vec = []
    for digito in seq:
        vec.append(digito)
    n = len(vec)
    if   vec[0] == '+':
        psi = (1/sqrt(2))*(cb(dim, 0) + cb(dim, 1))
    elif vec[0] == '-':
        psi = (1/sqrt(2))*(cb(dim, 0) - cb(dim, 1))
    elif vec[0] == 'r':
        psi = (1/sqrt(2))*(cb(dim, 0) + 1j*cb(dim, 1))
    elif vec[0] == 'l':
        psi = (1/sqrt(2))*(cb(dim, 0) - 1j*cb(dim, 1))
    elif vec[0] == 'P':
        psi = (1/sqrt(2))*(tp(cb(dim, 0), cb(dim, 0)) + tp(cb(dim, 1),cb(dim, 1)))
    elif vec[0] == 'Q':
        psi = (1/sqrt(2))*(tp(cb(dim, 0), cb(dim, 0)) - tp(cb(dim, 1),cb(dim, 1)))
    elif vec[0] == 'R':
        psi = (1/sqrt(2))*(tp(cb(dim, 0), cb(dim, 1)) + tp(cb(dim, 1),cb(dim, 0)))
    elif vec[0] == 'S':
        psi = (1/sqrt(2))*(tp(cb(dim, 0), cb(dim, 1)) - tp(cb(dim, 1),cb(dim, 0)))
    else:
        psi = cb(dim, int(vec[0]))
    for j in range(1,n):
        if   vec[j] == '+':
            psi = (1/sqrt(2))*(tp(psi,cb(dim, 0)) + tp(psi,cb(dim, 1)))
        elif vec[j] == '-':
            psi = (1/sqrt(2))*(tp(psi,cb(dim, 0)) - tp(psi,cb(dim, 1)))
        elif vec[j] == 'r':
            psi = (1/sqrt(2))*(tp(psi,cb(dim, 0)) + 1j*tp(psi,cb(dim, 1)))
        elif vec[j] == 'l':
            psi = (1/sqrt(2))*(tp(psi,cb(dim, 0)) - 1j*tp(psi,cb(dim, 1)))
        elif vec[j] == 'P':
            psi = (1/sqrt(2))*tp(psi,(tp(cb(dim, 0), cb(dim, 0)) + tp(cb(dim, 1),cb(dim, 1))))
        elif vec[j] == 'Q':
            psi = (1/sqrt(2))*tp(psi,(tp(cb(dim, 0), cb(dim, 0)) - tp(cb(dim, 1),cb(dim, 1))))
        elif vec[j] == 'R':
            psi = (1/sqrt(2))*tp(psi,(tp(cb(dim, 0), cb(dim, 1)) + tp(cb(dim, 1),cb(dim, 0))))
        elif vec[j] == 'S':
            psi = (1/sqrt(2))*tp(psi,(tp(cb(dim, 0), cb(dim, 1)) - tp(cb(dim, 1),cb(dim, 0))))
        else:
            psi = tp(psi,cb(dim, int(vec[j])))
    if mtype == 1:
        return transpose(psi).as_mutable()
    return psi

# Generic density matrix ($\rho$)

In [45]:
"""
Returns a symbolic generic rho
n = column dimension
Returns a symbolic generic rho
square matrix
n = row/column levels
"""
def rho_g(n, word='0'):
    num_digits = len(str(n**2 - 1))
    A = zeros(n,n)
    l = 0
    if word == '0':
        rho = sympy.symbols(['rho_{:0{}}'.format(i, num_digits) for i in range(n**2)])
        for j in range(0,n):
            for k in range(0,n):
                    A[j,k] = rho[l]
                    l += 1
        
    else: 
        rho = sympy.symbols(['{}{:0{}}'.format(word, i, num_digits) for i in range(n**2)])
        for j in range(0,n):
            for k in range(0,n):
                    A[j,k] = rho[l]
                    l += 1
    return A

In [1]:
"""
Returns a symbolic generic rho with indices in binary form
square matrix
n = row/column levels
"""
def rho_bin(n, word='0'):
    if word == '0':
        rho = [sympy.symbols('rho_{}'.format(format(i, '0{}b'.format(int(math.ceil(log(n**2, 2))))))) for i in range(n**2)]
        A = zeros(n,n)
        l = 0
        for j in range(0,n):
            for k in range(0,n):
                    A[j,k] = rho[l]
                    l += 1
    else:
        rho = [sympy.symbols('{}_{}'.format(word, format(i, '0{}b'.format(int(math.ceil(log(n**2, 2))))))) for i in range(n**2)]
        A = zeros(n,n)
        l = 0
        for j in range(0,n):
            for k in range(0,n):
                    A[j,k] = rho[l]
                    l += 1
    return A

# Dirac notation

In [51]:
"""
Return Psi or rho in bra-ket notation
Sent Psi - return Psi in bra-ket notation
Send rho - return rho in bra-ket notation
"""
def mbk(matrix, dim=2):
    """
    Args:
        matrix (sympy.matrices.dense.MutableDenseMatrix): The input must be a sympy array.
                                            It can be a state vector or a density matrix

    Returns:
        Prints the matrix in Dirac notation
    """
    def convert_dim(x, dim, min_digits):
        """
        Helper function that checks how many digits the bra-ket will 
        have based on the dimension and size of the input matrix
        """
        digits = '0123456789'[:dim]
        result = ''
        while x > 0 or len(result) < min_digits:
            x, digit = divmod(x, dim)
            result = digits[digit] + result
        return result
    pos = []
    pos_bin = []
    val = []
    if isinstance(matrix, sympy.matrices.dense.MutableDenseMatrix):
        n_linhas, n_colunas = matrix.shape
        """
        If the row == 1 the notation will be Bra, if the column == 1 the notation will be Ket and if 
        it doesn't fit in any of these cases, then it's because it's a density matrix and will be handled by 'else'
        """
        if n_linhas == 1:
            Psi = 0
            x = len(matrix)
            for i in range(x):
                if matrix[i] != 0:
                    val.append(matrix[i])
                    pos.append(i)
                    pos_bin.append(convert_dim(i, dim, int(math.ceil(math.log(x)/math.log(dim)))))
            for i in range(len(val)):
                Psi = val[i] * Bra(pos_bin[i]) + Psi
            return simplify(Psi)
        if n_colunas == 1:
            Psi = 0
            x = len(matrix)
            for i in range(x):
                if matrix[i] != 0:
                    val.append(matrix[i])
                    pos.append(i)
                    pos_bin.append(convert_dim(i, dim, int(math.ceil(math.log(x)/math.log(dim)))))
            for i in range(len(val)):
                Psi = val[i] * Ket(pos_bin[i]) + Psi
            return Psi
        else:
            m, n = matrix.shape
            rho = 0
            for i in range(m):
                for j in range(n):
                    if matrix[i, j] != 0:
                        val.append(matrix[i, j])
                        pos.append((i, j))
                        pos_bin.append((convert_dim(i, dim, int(math.ceil(math.log(m)/math.log(dim)))),\
                                             convert_dim(j, dim, int(math.ceil(math.log(m)/math.log(dim))))))
            for i in range(len(val)):
                rho = val[i] * (Ket(pos_bin[i][0])) * (Bra(pos_bin[i][1])) + rho
            return rho

# Composite systems

In [ ]:
def comp_sys(*psi):
    """
    *psi (sympy.matrices.dense.MutableDenseMatrix): The input must be 'n' subsystems as sympy array.
                                                          Must be a state vectors or list of state vector.
                                                          Se examples for more details.
    Returns:
        Returns the tensor product of the state vectors in the order they are entered into the function.
    """
    if (psi).__class__ == tuple and len(psi)==1:
        order = []
        for j in range(len(*psi)):
            order.append(psi[0][j])
        return Matrix(reduce(tp, [x for x in order]))
    else:
        return Matrix(reduce(tp, [x for x in psi]))

#  Changing the position of the qubit: `changeSS(psi, pos_i, pos_f)`

In [2]:
def changeSS(psi, pos_i, pos_f):
    """
    Changes the subsystem from a given position (pos_i) to another position (pos_f).
    Args:
        matrix (sympy.matrices.dense.MutableDenseMatrix): The input must be a sympy array.
                                                          Must be a state vector.
        pos_i (int): n-1, n-2,..., 1, 0.
        pos_f (int): n-1, n-2,..., 1, 0.
            where n is the number of subsystems.
    Returns:
        Returns the state vector with the qubit in the desired position.
    """
    def move_bit(binary, pos_i, pos_f, n):
        characters = list(binary)
        if n-1-pos_i == n-1-pos_f:
            return binary
        bit = characters.pop(n-1-pos_i)
        characters.insert(n-1-pos_f, bit)
        binm = ''.join(characters)
        return binm
    lenp = len(psi)
    n = math.ceil(math.log(lenp, 2))
    psi_f = [0]*lenp
    for j in range(lenp):
        psi_f[int(move_bit('{:0{}b}'.format(j, n), pos_i, pos_f, n), 2)] = psi[j]
    return Matrix(psi_f)

# Tensor products for quantum logic gates: `tp_gate(n, gate, ssystem)`

In [30]:
def tp_gate(n, gate, *ssystem):
    """
    Adjusts the matrix for the subsystems that will act the quantum logic gate (QLG).
    Args:
        n (int): é a quantidade de subsistemas envolvidos
        gate (sympy.matrices.dense.MutableDenseMatrix): 2x2 dimension QLG matrix
        ssystem (int): subsystems that QLG will act (subsystems position in the bra-ket).
                        0 ≤ ssystem ≤ n-1
        tconj (int): default is 0. If it is 1, the return will be the conjugate transposed matrix.

    Returns:
        Matrix adjusted to act the QLG in a certain subsystem considering that we have n subsystems
    """
    if len(ssystem) == 1 and isinstance(ssystem[0], (list, tuple)):
        lst = list(ssystem[0])
    else:
        lst = list(ssystem)
    lst.sort(reverse=False)
    order = []
    for j in range(n-1,-1,-1):
        if j in lst:
            order.append(gate)
        else:
            order.append(sym.eye(2))
    return tp(*order)

## Acting a quantum logic gate in a state: `gatep(gate, psi, ssystem)`

In [35]:
def gatep(gate, psi, ssystem):
    n = int(log(len(psi), 2))
    if n == 1:
        return gate*psi
    tpgate = tp_gate(n, gate, ssystem)
    return tpgate * psi

# Tensor products for controlled quantum logic gates:
`tp_ctrl(n, gate, ctrl_act, target)`

In [129]:
def tp_ctrl(n, gate, ctrl_act, target):
    """
    Adjusts the matrix for the subsystems that will act the controlled quantum logic gate (CQLG).
    
    Args:
        n (int): é a quantidade de subsistemas envolvidos
        gate (sympy.matrices.dense.MutableDenseMatrix): 2x2 2ension quantum logic gate array that you want
                                                        to be controlled by other subsystems
        *ctrl (list): is the controls of CQLG - 0 ≤ ctrl ≤ n-1. Example with multi controls: (2,1), (1,3)
        target (int): is the target of CQLG - 0 ≤ target ≤ n-1
                        target nedds to be different of ctrl
        act (int): CQLG enable bit.
                    1: 1 to be the enable bit (default)
                    0: 0 to be the enable bit

    Returns:
        Matrix adjusted to act the QLG in a certain subsystem considering that we have n subsystems
    """
    order = []
    if len(ctrl_act) >= 2 and isinstance(ctrl_act[1], int):
        ctrl_lst = []
        if isinstance(ctrl_act[0], int):
            ctrl_lst = ctrl_act[0:-1]
            act = [ctrl_act[-1]]*len(ctrl_lst)
        else:
            act = [ctrl_act[1]]*len(ctrl_act[0])
            ctrl_lst = list(ctrl_act[0])
    else:
        ctrl_lst, act = [], []
        for el in ctrl_act:
            ctrl_lst.append(el[0])
            act.append(el[1])
    #
    # Verifications
    #
    for el in target:
        if el in ctrl_lst:
            return print('Target qubit cannot be control either')
    if len(target) + len(ctrl_lst) > n:
        return print('There are more targets and controls than subsystems')
    for val in ctrl_lst:
        if val >= n or val < 0:
            return print('The control is outside the allowed limits')
    for val in target:
        if val >= n or val < 0:
            return print('The target is outside the allowed limits')
    #
    #
    #
    for i in range(n-1,-1,-1):
        if i in target:
            order.append((gate, sym.eye(2)))
        elif i in ctrl_lst:
            if act[ctrl_lst.index(i)] == 0:
                order.append((proj(cb(2, 0)), proj(cb(2, 1))))
            if act[ctrl_lst.index(i)] == 1:
                order.append((proj(cb(2, 1)), proj(cb(2, 0))))
        else:
            order.append((sym.eye(2),sym.eye(2)))
    CGate = Matrix(reduce(tp, [x[0] for x in order]) + reduce(tp, [x[1] for x in order]))
    for j in range(CGate.rows):
        flag = 0
        for k in range(CGate.cols):
            if CGate[j, k] != 0:
                flag = 1
                break
        if flag == 0:
            CGate[j,j] = 1
    return CGate

## Acting a controlled quantum logic gate in a state: `ctrlp(gate, psi, ctrl_act, target)`

In [ ]:
def ctrlp(gate, psi, ctrl_act, target):
    """    
    Args:
        matrix (sympy.matrices.dense.MutableDenseMatrix): The input must be a sympy array.
                                                          Must be a state vector.
        ctrl (int): is the control of CQLG - 0 ≤ ctrl ≤ n-1
        target (int): is the target of CQLG - 0 ≤ target ≤ n-1
        act (int): CQLG enable bit.
                    1: 1 to be the enable bit (default)
                    0: 0 to be the enable bit

    Returns:
        Returns the state vector evolved by CQLG
    """
    n = int(log(len(psi), 2))
    ctrl_gate = tp_ctrl(n, gate, ctrl_act, target)
    return ctrl_gate * psi